In [197]:
import maup # mggg's library for proration, see documentation here: https://github.com/mggg/maup
import pandas as pd # standard python data library
import geopandas as gp # the geo-version of pandas
import numpy as np 
import os
import fiona
from statistics import mean, median
from pandas import read_csv
gp.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw' #To load KML files
import string
import xml.etree.ElementTree as et

In [198]:
vest_mo_18 = gp.read_file("./raw-from-source/VEST/mo_2018/mo_2018.shp")

## Election Data Sources

Election results from the Secretary of State's office via OpenElections (https://github.com/openelections/openelections-data-mo/tree/master/2018). Absentee, provisional, and mail ballots were reported countywide in nearly every county; these were distributed by candidate to precincts based on their share of the precinct-level reported vote.

- G18USSRHAW - Josh Hawley (Republican Party)  
- G18USSDMCC - Claire McCaskill (Democratic Party)  
- G18USSLCAM - Japheth Campbell (Libertarian Party)  
- G18USSGCRA - Jo Crain (Green Party)  
- G18USSIODE - Craig O'Dear (Independent)  
  
  
- G18AUDRMCD - Saundra McDowell (Republican Party)  
- G18AUDDGAL - Nicole Galloway (Democratic Party)  
- G18AUDLOTO - Sean O'Toole (Libertarian Party)  
- G18AUDGFIT - Don Fitz (Green Party)  
- G18AUDCLUE - Jacob Luetkemeyer (Constitution Party)  


In [199]:
mo_elections = pd.read_csv("./raw-from-source/Open_Elections/20181106__mo__general__precinct.csv")
candidate_list = ['Josh Hawley','Claire McCaskill','Japheth Campbell','Jo Crain',
 "Craig O'Dear",'Saundra McDowell','Nicole Galloway',"Sean O'Toole",'Don Fitz','Jacob Luetkemeyer']
office_list = ['U.S. Senate', 'State Auditor']
mo_elections["candidate"] = mo_elections["candidate"].str.strip()
mo_elections = mo_elections[mo_elections["office"].isin(office_list)]
mo_elections = mo_elections[mo_elections["candidate"].isin(candidate_list)]


In [200]:
fips_file = pd.read_csv("./raw-from-source/FIPS/US_FIPS_Codes.csv")
fips_file = fips_file[fips_file["State"]=="Missouri"]
fips_file["FIPS County"] = fips_file["FIPS County"].astype(str)
fips_file["FIPS County"] = fips_file["FIPS County"].str.zfill(3)
#print(fips_file[~fips_file["County Name"].isin(mo_elections["county"])]["County Name"].unique())
fips_dict = dict(zip(fips_file['County Name'], fips_file['FIPS County']))
mo_changes = {'De Kalb':'Dekalb','Kansas City':'Jackson','McDonald':'Mcdonald',
              'St. Charles':'St Charles','St. Clair':'St Clair',
              'Ste. Genevieve':'Ste Genevieve','St. Francois':'St Francois',
              'St. Louis City':'St Louis City','St. Louis County':'St Louis'}
mo_elections['county'] = mo_elections['county'].map(mo_changes).fillna(mo_elections['county'])
mo_elections['COUNTYFP'] = mo_elections['county'].map(fips_dict).fillna(mo_elections['county'])
mo_elections["unique_ID"]=mo_elections["COUNTYFP"]+mo_elections["precinct"]

In [201]:
print(sum(mo_elections[mo_elections["county"]=="Jackson"]["votes"]))
print(mo_elections[mo_elections["county"]=="Jackson"])

523967
        county       precinct         office  district party  \
22552  Jackson      B1 01, 02    U.S. Senate       NaN   REP   
22553  Jackson      B1 03, 09    U.S. Senate       NaN   REP   
22554  Jackson          B1 04    U.S. Senate       NaN   REP   
22555  Jackson          B1 05    U.S. Senate       NaN   REP   
22556  Jackson  B1 06, 07, 08    U.S. Senate       NaN   REP   
...        ...            ...            ...       ...   ...   
33897  Jackson        W26-P10  State Auditor       NaN   NaN   
33898  Jackson        W26-P10  State Auditor       NaN   NaN   
33899  Jackson        W26-P10  State Auditor       NaN   NaN   
33900  Jackson        W26-P10  State Auditor       NaN   NaN   
33901  Jackson        W26-P10  State Auditor       NaN   NaN   

               candidate  votes COUNTYFP         unique_ID  
22552        Josh Hawley    516      095      095B1 01, 02  
22553        Josh Hawley    315      095      095B1 03, 09  
22554        Josh Hawley    256      095 

In [202]:
sum_tot = 0
for i in col_list:
    sum_tot += sum(vest_mo_18[vest_mo_18["COUNTYFP"]=="095"][i])
print(sum_tot)

523967


In [203]:
mo_elections_pivot = pd.pivot_table(mo_elections,index=["unique_ID","COUNTYFP","precinct"],columns=["candidate"],values=['votes'],aggfunc=sum)
mo_elections_pivot = mo_elections_pivot.fillna(0)
mo_elections_pivot.columns = mo_elections_pivot.columns.droplevel(0)
mo_elections_pivot.reset_index(drop=False,inplace=True)
mo_elections_pivot = mo_elections_pivot.rename(columns={'Claire McCaskill':"G18USSDMCC","Craig O'Dear": "G18USSIODE",'Don Fitz':"G18AUDGFIT",'Jacob Luetkemeyer':"G18AUDCLUE",'Japheth Campbell':"G18USSLCAM",'Jo Crain':"G18USSGCRA",'Josh Hawley':"G18USSRHAW",'Nicole Galloway':"G18AUDDGAL",'Saundra McDowell':"G18AUDRMCD","Sean O'Toole":"G18AUDLOTO"})

In [204]:
sum_tot = 0
for i in col_list:
    sum_tot += sum(mo_elections_pivot[mo_elections_pivot["COUNTYFP"]=="095"][i])
print(sum_tot)

523967.0


In [205]:
remove_list = ["ABSENTEE","PROVISIONAL","FEDERAL","FEDERAL/STATE","INTRASTATE","WRITE-IN","CENTRAL POLLING LOCATION","CENTRAL POLL","ABSENTEE1","ABSENTEE2","VOTE CENTER","CENTRAL POLLING LOCATION: MSU","INTRASTATE/NEW RESIDENT","WRITE-INS","MULTI-PRECINCT COLLECTION","INTRASTATE NEW RESIDENT","MULTI-PRECINCT COLLECTION","ABSENTEE, PROVISIONAL & CENTRAL"]
mo_elections_countywide = mo_elections_pivot[(mo_elections_pivot["precinct"].isin(remove_list)) | (mo_elections_pivot["unique_ID"]=='019CENTRAL')]
mo_elections_pivot = mo_elections_pivot[~(mo_elections_pivot["precinct"].isin(remove_list)) | (mo_elections_pivot["unique_ID"]=='019CENTRAL')]
print(mo_elections_countywide.head())
print(mo_elections_pivot.head())
countywide_totals = mo_elections_countywide.groupby(["COUNTYFP"]).sum()
precinct_totals = mo_elections_pivot.groupby(["COUNTYFP"]).sum()
data_to_add = pd.merge(precinct_totals,countywide_totals,on=['COUNTYFP'],how='left',indicator=False)
data_to_add.reset_index(inplace=True,drop=False)

candidate    unique_ID COUNTYFP  precinct  G18USSDMCC  G18USSIODE  G18AUDGFIT  \
0          001ABSENTEE      001  ABSENTEE       428.0        11.0         5.0   
2           001FEDERAL      001   FEDERAL         0.0         0.0         0.0   
12         003ABSENTEE      003  ABSENTEE       179.0         6.0         1.0   
24         005ABSENTEE      005  ABSENTEE        30.0         1.0         1.0   
38         007ABSENTEE      007  ABSENTEE       174.0         7.0         0.0   

candidate  G18AUDCLUE  G18USSLCAM  G18USSGCRA  G18USSRHAW  G18AUDDGAL  \
0                33.0         9.0         5.0       601.0       438.0   
2                 0.0         0.0         0.0         2.0         0.0   
12               29.0         4.0         4.0       309.0       193.0   
24                5.0         1.0         1.0        98.0        36.0   
38               19.0         1.0         2.0       302.0       199.0   

candidate  G18AUDRMCD  G18AUDLOTO  
0               527.0        17.0  
2 

In [206]:
#print(precinct_totals[precinct_totals["candidate"]=="095"])
print(data_to_add[data_to_add["COUNTYFP"]=="095"])
sum_tot = 0
for i in col_list:
    x = i+"_x"
    y = i+"_y"
    sum_tot += sum(data_to_add[data_to_add["COUNTYFP"]=="095"][x])
    sum_tot += sum(data_to_add[data_to_add["COUNTYFP"]=="095"][y])
print(sum_tot)

candidate COUNTYFP  G18USSDMCC_x  G18USSIODE_x  G18AUDGFIT_x  G18AUDCLUE_x  \
47             095      158375.0        3169.0        2085.0        4102.0   

candidate  G18USSLCAM_x  G18USSGCRA_x  G18USSRHAW_x  G18AUDDGAL_x  \
47               2601.0        1423.0       87638.0      161831.0   

candidate  G18AUDRMCD_x  ...  G18USSDMCC_y  G18USSIODE_y  G18AUDGFIT_y  \
47              75385.0  ...        5642.0         117.0          66.0   

candidate  G18AUDCLUE_y  G18USSLCAM_y  G18USSGCRA_y  G18USSRHAW_y  \
47                192.0          83.0          61.0        5105.0   

candidate  G18AUDDGAL_y  G18AUDRMCD_y  G18AUDLOTO_y  
47               5685.0        4622.0         180.0  

[1 rows x 21 columns]
523967.0


In [207]:


joined = pd.merge(mo_elections_pivot,data_to_add,on=['COUNTYFP'],how='left',indicator=False)
joined = joined.fillna(0)
col_list = ["G18USSDMCC","G18USSIODE","G18AUDGFIT","G18AUDCLUE","G18USSLCAM","G18USSGCRA","G18USSRHAW","G18AUDDGAL","G18AUDRMCD","G18AUDLOTO"]
for i in col_list:
    precinct_totals = i + "_x"
    countywide_reported = i+"_y"
    joined[i]=round(joined[i]+((joined[i]*joined[countywide_reported])/joined[precinct_totals]),0)
joined = joined [["unique_ID","COUNTYFP","precinct","G18USSDMCC","G18USSIODE","G18AUDGFIT","G18AUDCLUE","G18USSLCAM","G18USSGCRA","G18USSRHAW","G18AUDDGAL","G18AUDRMCD","G18AUDLOTO"]]
matcher_increase = pd.read_csv("./matcher_increase.csv")
print(matcher_increase.head())
new_name_dict = dict(zip(matcher_increase['source_names'], matcher_increase['vest_names']))
joined["unique_ID"] = joined["unique_ID"].str.strip()
joined["unique_ID"] = joined["unique_ID"].map(new_name_dict).fillna(joined["unique_ID"])

     source_names      vest_names
0             197            1907
1             198            1908
2     001BRASHEAR     001Brashear
3  001NORTHEAST 5  001Northeast 5
4  001NORTHEAST 6  001Northeast 6


In [208]:
sum_tot = 0
for i in col_list:
    sum_tot += sum(joined[joined["COUNTYFP"]=="095"][i])
print(sum_tot)

523749.0


- G18USSRHAW - Josh Hawley (Republican Party)  
- G18USSDMCC - Claire McCaskill (Democratic Party)  
- G18USSLCAM - Japheth Campbell (Libertarian Party)  
- G18USSGCRA - Jo Crain (Green Party)  
- G18USSIODE - Craig O'Dear (Independent)  
  
  
- G18AUDRMCD - Saundra McDowell (Republican Party)  
- G18AUDDGAL - Nicole Galloway (Democratic Party)  
- G18AUDLOTO - Sean O'Toole (Libertarian Party)  
- G18AUDGFIT - Don Fitz (Green Party)  
- G18AUDCLUE - Jacob Luetkemeyer (Constitution Party)  

In [209]:
vest_mo_18["unique_ID"]=vest_mo_18["COUNTYFP"]+vest_mo_18["NAME"]
vest_empty = vest_mo_18[(vest_mo_18['G18USSRHAW']==0) & (vest_mo_18['G18USSDMCC']==0) &
              (vest_mo_18['G18USSLCAM']==0) & (vest_mo_18['G18USSGCRA']==0) &
              (vest_mo_18['G18USSIODE']==0) & (vest_mo_18['G18AUDRMCD']==0) &
              (vest_mo_18['G18AUDDGAL']==0) & (vest_mo_18['G18AUDLOTO']==0) &
              (vest_mo_18['G18AUDGFIT']==0) & (vest_mo_18['G18AUDCLUE']==0)]["unique_ID"]

new_precincts = pd.DataFrame(columns=joined.columns)
new_precincts["unique_ID"]=vest_empty
col_list = ["G18USSDMCC","G18USSIODE","G18AUDGFIT","G18AUDCLUE","G18USSLCAM","G18USSGCRA","G18USSRHAW","G18AUDDGAL","G18AUDRMCD","G18AUDLOTO"]
for i in col_list:
    new_precincts[i]=0
joined = joined.append(new_precincts)

In [210]:
print(vest_mo_18.head())

final_join = pd.merge(vest_mo_18,joined,how="outer",on="unique_ID",indicator=True)
print(final_join["_merge"].value_counts())
final_join[final_join["_merge"]=="right_only"].to_csv("./election_only.csv")
final_join[final_join["_merge"]=="left_only"].to_csv("./VEST_only.csv")

  STATEFP COUNTYFP           NAME  G18USSRHAW  G18USSDMCC  G18USSLCAM  \
0      29      189          MR032          77          26           0   
1      29      189  CHE044/LAF001         254         288           2   
2      29      201       Commerce         215          89           1   
3      29      043    West Finley        1254         504          34   
4      29      189  MID036/MID048          48         257           1   

   G18USSGCRA  G18USSIODE  G18AUDRMCD  G18AUDDGAL  G18AUDLOTO  G18AUDGFIT  \
0           0           0          57          42           0           0   
1           2           2         227         303           7           0   
2           0           0         197          89           5           0   
3          14          38        1056         651          67          15   
4           4           9          37         268           5           4   

   G18AUDCLUE                                           geometry  \
0           2  POLYGON ((-90.5

In [211]:
print(final_join[final_join["_merge"]=="both"].head())


  STATEFP COUNTYFP_x           NAME  G18USSRHAW_x  G18USSDMCC_x  G18USSLCAM_x  \
0      29        189          MR032          77.0          26.0           0.0   
1      29        189  CHE044/LAF001         254.0         288.0           2.0   
2      29        201       Commerce         215.0          89.0           1.0   
3      29        043    West Finley        1254.0         504.0          34.0   
4      29        189  MID036/MID048          48.0         257.0           1.0   

   G18USSGCRA_x  G18USSIODE_x  G18AUDRMCD_x  G18AUDDGAL_x  ...  G18USSIODE_y  \
0           0.0           0.0          57.0          42.0  ...           0.0   
1           2.0           2.0         227.0         303.0  ...           2.0   
2           0.0           0.0         197.0          89.0  ...           0.0   
3          14.0          38.0        1056.0         651.0  ...          38.0   
4           4.0           9.0          37.0         268.0  ...           9.0   

   G18AUDGFIT_y  G18AUDCLUE_y G1

In [216]:
def validater_row (df, column_List):
    matching_rows = 0
    different_rows = 0
    diff_list=[]
    diff_values = []
    max_diff = 0
    
    for j in range(0,len(df.index)):
        same = True
        for i in column_List:
            left_Data = i + "_x"
            right_Data = i + "_y"
            diff = abs(df.iloc[j][left_Data]-df.iloc[j][right_Data])
            if(diff != 0):
                diff_values.append(abs(diff))
                same = False
                if(np.isnan(diff)):
                    print("NaN value at diff is: ", df.iloc[j]['unique_ID'])
                if (diff>max_diff):
                    max_diff = diff
                    print("New max diff is: ", str(max_diff))
                    print(df.iloc[j]['unique_ID'])
        if(same != True):
            different_rows +=1
            diff_list.append(df.iloc[j]['unique_ID'])
        else:
            matching_rows +=1
    print("There are ", len(df.index)," total rows")
    print(different_rows," of these rows have election result differences")
    print(matching_rows," of these rows are the same")
    print("")
    print("The max difference between any one shared column in a row is: ", max_diff)
    if(len(diff_values)!=0):
        print("The average difference is: ", str(sum(diff_values)/len(diff_values)))
    count_big_diff = len([i for i in diff_values if i > 10])
    print("There are ", str(count_big_diff), "precinct results with a difference greater than 5")
    diff_list.sort()
    print(diff_list)

In [217]:
col_list = ["G18USSDMCC","G18USSIODE","G18AUDGFIT","G18AUDCLUE","G18USSLCAM","G18USSGCRA","G18USSRHAW","G18AUDDGAL","G18AUDRMCD","G18AUDLOTO"]
validater_row(final_join[final_join["_merge"]=="both"],col_list)


New max diff is:  1.0
043West Finley
New max diff is:  42.0
095KC 903/904/906
New max diff is:  60.0
095KC 603/604/605/606/617/620
New max diff is:  62.0
095KC 403/404/405/411
New max diff is:  68.0
095KC 807/808/810/811
There are  3110  total rows
1288  of these rows have election result differences
1822  of these rows are the same

The max difference between any one shared column in a row is:  68.0
The average difference is:  5.598517251211862
There are  615 precinct results with a difference greater than 5
['001Brashear', '001Northeast 5', '001Novinger', '001Rural Benton', '001Southeast 2', '001Southwest 1', '003John Glenn/Country Club/Beeler', '003Lincoln', '003Rochester', '005Rock Port', '005Tarkio', '007Community', '007Martinsburg', '007Mexico No. 1', '007Mexico No. 2/3/4', '007Mexico No. 5', '007Mexico No. 6', '007Rush Hill/Benton City', '007South Fork', '009Cassville City', '009Cassville Rural/Mineral', '009Exeter', '009McDowell/Purdy', '009Ozark/Crane Creek', '009Roaring River

In [53]:
print(final_join[final_join["unique_ID"]=='0772A/2B'])

     STATEFP COUNTYFP_x   NAME  G18USSRHAW_x  G18USSDMCC_x  G18USSLCAM_x  \
2690      29        077  2A/2B         236.0         488.0          10.0   

      G18USSGCRA_x  G18USSIODE_x  G18AUDRMCD_x  G18AUDDGAL_x  ...  \
2690           6.0           3.0         182.0         509.0  ...   

      G18USSIODE_y  G18AUDGFIT_y  G18AUDCLUE_y G18USSLCAM_y G18USSGCRA_y  \
2690          43.0          25.0          33.0         58.0         17.0   

     G18USSRHAW_y G18AUDDGAL_y  G18AUDRMCD_y  G18AUDLOTO_y  _merge  
2690       1445.0       1266.0        1212.0          89.0    both  

[1 rows x 28 columns]


In [46]:

def validater_row_vect(df, county_column, join_column, column_List):
    matching_rows = 0
    different_rows = 0
    
    county_join_cols = (df[join_column].astype('str')).to_numpy()
    
    # it's because before, we were checking that a row was entirely consistent
    # here, we are double checking a lot
    # so, let's just keep one track of the rows that are messed up
    
    wrong_idxs = np.zeros(len(df))
    for i in column_List:
        left_data = df[i + "_x"].to_numpy()
        right_data = df[i + "_y"].to_numpy()
        
        local_idxs = np.where(left_data != right_data)
        wrong_idxs[local_idxs] = 1
        #print("Wrong idxs", wrong_idxs)
        
    # we are close, we get the same result, but are double adding lots of rows
    different_rows += np.sum(wrong_idxs)
    matching_rows += len(df) - different_rows
    
    diff_list = county_join_cols[np.where(wrong_idxs == 1)]
            
    print("There are ", len(df.index)," total rows")
    print(f"{int(different_rows)} of these rows have election result differences")
    print(f"{int(matching_rows)} of these rows are the same")
    print(diff_list)
    print("")

    

col_list = ["G18USSDMCC","G18USSIODE","G18AUDGFIT","G18AUDCLUE","G18USSLCAM","G18USSGCRA","G18USSRHAW","G18AUDDGAL","G18AUDRMCD","G18AUDLOTO"]
validater_row_vect(final_join[final_join["_merge"]=="both"],"unique_ID","unique_ID",col_list)

There are  2852  total rows
1073 of these rows have election result differences
1779 of these rows are the same
['043West Finley' '165Barry East 34' '123Marquand' ...
 '175Jacksonville/Levicks Mill/Cairo' '147Green' '147Nodaway']



In [50]:

print(final_join[final_join["unique_ID"]=='043West Finley'])
col_list = ["G18USSDMCC","G18USSIODE","G18AUDGFIT","G18AUDCLUE","G18USSLCAM","G18USSGCRA","G18USSRHAW","G18AUDDGAL","G18AUDRMCD","G18AUDLOTO"]
for i in col_list:
    x = i+"_x"
    y = i+"_y"
    print(final_join[final_join["unique_ID"]=='043West Finley'][x])
    print(final_join[final_join["unique_ID"]=='043West Finley'][y])

  STATEFP COUNTYFP_x         NAME  G18USSRHAW_x  G18USSDMCC_x  G18USSLCAM_x  \
3      29        043  West Finley        1254.0         504.0          34.0   

   G18USSGCRA_x  G18USSIODE_x  G18AUDRMCD_x  G18AUDDGAL_x  ...  G18USSIODE_y  \
3          14.0          38.0        1056.0         651.0  ...          38.0   

   G18AUDGFIT_y  G18AUDCLUE_y G18USSLCAM_y G18USSGCRA_y G18USSRHAW_y  \
3          15.0          20.0         34.0         13.0       1254.0   

  G18AUDDGAL_y  G18AUDRMCD_y  G18AUDLOTO_y  _merge  
3        651.0        1056.0          67.0    both  

[1 rows x 28 columns]
3    504.0
Name: G18USSDMCC_x, dtype: float64
3    504.0
Name: G18USSDMCC_y, dtype: float64
3    38.0
Name: G18USSIODE_x, dtype: float64
3    38.0
Name: G18USSIODE_y, dtype: float64
3    15.0
Name: G18AUDGFIT_x, dtype: float64
3    15.0
Name: G18AUDGFIT_y, dtype: float64
3    20.0
Name: G18AUDCLUE_x, dtype: float64
3    20.0
Name: G18AUDCLUE_y, dtype: float64
3    34.0
Name: G18USSLCAM_x, dtype: float64

## Precinct Shapefile Sources

The documentation mentions 4 different sources for shapefiles:

1. Precinct shapefiles primarily from the U.S. Census Bureau's 2020 Redistricting Data Program Phase 2 release.  
2. Monroe County comes from the 2010 Census VTD release.  
3. Platte County comes from the 2020 Census VTD release.  
4. The following counties instead used maps or shapefiles sourced from the respective county governments: Audrain, Bates, Caldwell, Callaway, Camden, Cape Girardeau, Cedar, Cooper, Franklin, Greene, Jackson, Jasper, Jefferson, Laclede, Lafayette, Lawrence, Marion, McDonald, Miller, Nodaway, Osage, Ozark, Pemiscot, Pike, Randolph, Scott, Ste. Genevieve, Texas, Warren, Washington, Worth, Wright.  

### 1. U.S. Census Bureau's 2020 Redistricting Data Program Phase 2 release
> Precinct shapefiles primarily from the U.S. Census Bureau's 2020 Redistricting Data Program Phase 2 release.

https://www.census.gov/geo/partnerships/pvs/partnership19v2/st29_mo.html

In [ ]:
#When downloading from the Census redistricing data program, these use a FIPS code to identify counties

fips_file["unique_ID"] =  "29" + fips_file["FIPS County"]
fips_codes = fips_file["unique_ID"].tolist()

#Combine all the data from separate files into one
li = []
for i in fips_codes:
    ref = "./raw-from-source/Census/2020_Phase_2/partnership_shapefiles_19v2_"
    file_ref = ref+i+"/PVS_19_v2_vtd_"+i+".shp"
    file_prev = gp.read_file(file_ref)
    #print(file_prev.shape)
    li.append(file_prev)
shapefiles_census = pd.concat(li, axis=0, ignore_index=True)
print(shapefiles_census.shape)

In [ ]:
#print(shapefiles_census.head())
county_list = ["Audrain", "Bates","Caldwell","Callaway","Camden","Cape Girardeau","Cedar","Cooper","Franklin","Greene","Jackson","Jasper","Jefferson", 
               "Laclede", "Lafayette","Lawrence", "Marion", "Mcdonald","Miller","Monroe","Nodaway",
               "Osage", "Ozark", "Pemiscot","Platte", "Pike", "Randolph", "Scott","Ste Genevieve", "Texas", "Warren", "Washington", "Worth", "Wright"]
print(len(county_list))
fips_to_take = fips_file[fips_file["County Name"].isin(county_list)]
different_source = fips_to_take["FIPS County"].tolist()
print(different_source)

shapefiles_census = shapefiles_census[~shapefiles_census["COUNTYFP"].isin(different_source)]

In [ ]:
#Filter down election results to just these counties
print(joined.shape)
elections_census_shapes = joined[~joined["COUNTYFP"].isin(different_source)]
print(elections_census_shapes.shape)

#Create a unique ID
shapefiles_census["unique_ID"] = shapefiles_census["COUNTYFP"]+shapefiles_census["NAME"]

#Attempt to join on the "unique_ID" column
census_join = pd.merge(elections_census_shapes,shapefiles_census,how="outer",on="unique_ID",indicator=True)
print(census_join["_merge"].value_counts())
census_join[census_join["_merge"]=="left_only"].to_csv("./census_join/election.csv")
census_join[census_join["_merge"]=="right_only"].to_csv("./census_join/shapes.csv")

In [ ]:
print(shapefiles_census[shapefiles_census["COUNTYFP"].str[0:3]=="009"]["unique_ID"])
print(elections_census_shapes[elections_census_shapes["unique_ID"].str[0:3]=="009"]["unique_ID"])

In [1]:
precinct_merger(["003John Glenn A" "003Country Club","003John Glenn","003Beeler"],"003John Glenn/Country Club/Beeler")
precinct_merger(["003Platte West","003Rosendale","003Bolckow"],"003Platte West/Rosendale/Bolckow")
precinct_merger(["003Fillmore","003Clay"],"003Fillmore/Clay")
precinct_merger(["003Empire East","003Platte East"],"003Platte East/Empire East")
precinct_merger(["003Savannah Northeast","003Savannah Southeast"],"003Savannah Northeast/Southeast")
precinct_merger(["003Savannah Northwest","003Savannah Southwest"],"003Savannah Northwest/Southwest")
precinct_merger(["009McDonald","009Butterfield"],"009McDonald/Butterfield")
precinct_merger(["009Cassville Rural","009Mineral"],"009Cassville Rural/Mineral")
precinct_merger(["009Kings Prairie","009Pleasant Ridge"],"009Kings Prairie/Pleasant Ridge")
precinct_merger(["009Capps Creek"],"009Monett/Capps Creek")
precinct_merger(["009Crane Creek","009Ozark"],"009Ozark/Crane Creek")
precinct_merger(["009Purdy","009McDowell"],"009McDowell/Purdy")
precinct_merger(["009Ash","009Washburn"],"009Washburn/Ash")
precinct_merger(["015COLE"],"015Cole")
precinct_merger(["015ALEXANDER","015FRISTOE"],"015Fristoe/Alexander")
precinct_merger(["015LINDSEY"],"015Lindsey")
precinct_merger(["015TOM"],"015Tom")
precinct_merger(["015UNION"],"015Union")
precinct_merger(["015WHITE"],"015White")
precinct_merger(["015WILLIAMS"],"015Williams")

precinct_merger(["017Drum-Dongola"],"017Drum Dungola")
precinct_merger(["017Lutesville","017Marble Hill"],"017Lutesville/Marble Hill")

county_list = ["01909/11","01910/12","01915/40","01918/19","0191C/1G","0191E/6B","0191F/1J/1L","0191I/6F","0191K/2K","01922/45","01924/33","01925/34","01928/29","0192A/2G","0192B/2J","0192C/2F","0192L/20","01930/43","01931/32","01935/42","0193A/3G/3L","0193E/3H","0193I/3J","0193M/6J","0193N/41","0194A/4J","0194C/4D","0194F/4I/5A","0194H/4N","0194K/4M/37","0195C/5H","0195D/5G/5M","0195E/5P","0195I/5J","0195N/36/44","0196A/6L","0196C/6H","0196D/6K/6M","0196E/6I"]
#For those, split on the "/" and call precinct merger
county_list_2 = pd.read_csv("./189_columns.csv")
county_list_3 = list(county_list_2["val"])
print(county_list_3)

precinct_merger(["021Bible Baptist Temple"],"021Riverside Baptist Church")
precinct_merger(["033Carrollton Ward 3","033Carrollton Ward 2"],"033Carrollton Ward 2/3")
precinct_merger(["023Hill View"],"023Hillview")
precinct_merger(["033Prarie/Egypt"],"033Prairie/Egypt")
precinct_merger(["035Colemansville","035Ellsinore"],"035Colemansville/Ellsinore")
precinct_merger(["510ST 1-4"],"510STL 1-4")

call_list = {
    '03701 Sherman':[ ],
'03707 Drexel':[ ],
'03710 Grand River Rural':[ ],
'03711 Harrisonville Ward 1':[ ],
'03712 Harrisonville Ward 2':[ ],
'03713 Harrisonville Ward 3':[ ],
'03714 Harrisonville Ward 4':[ ],
'03716 East Lynne/18 Gunn City':[ ],
'03719 Polk':[ ],
'03702 Dayton/03 Garden City/15 Camp Branch/17 Index':[ ],
'03720 Pleasant Hill':[ ],
'03721 Pleasant Hill East':[ ],
'03722 Pleasant Hill':[ ],
'03723 Pleasant Hill Rural':[ ],
'03724 Peculiar Rural':[ ],
'03725 West Peculiar Rural':[ ],
'03726 West Peculiar City':[ ],
'03727 Union':[ ],
'03728 Big Creek Rural':[ ],
'03729 Winnebago':[ ],
'03730 Raymore Rural':[ ],
'03731 Raymore Ward 3':[ ],
'03732 Raymore Ward 2':[ ],
'03733 Raymore Ward 2':[ ],
'03734 Raymore Wards 3 & 4':[ ],
'03735 Raymore Ward 4':[ ],
'03736 Mt Pleasant Rural':[ ],
'03737 Pleasant Prairie':[ ],
'03738 Belton NE Ward 1':[ ],
'03739 Belton NE Ward 1':[ ],
'03704 Austin/05 Archie/06 Everett':[ ],
'03740 Belton NW Ward 2':[ ],
'03741 Belton NW Ward 2':[ ],
'03742 Belton SW Ward 3':[ ],
'03743 Belton SW Ward 3':[ ],
'03744 Belton SW Ward 4':[ ],
'03745 Belton SE Ward 4':[ ],
'03746 East Creek':[ ],
'03747 Raymore Ward 1':[ ],
'03708 West Dolan/09 Dolan':[ ],
    
#Make 043 go to only lowercase at the start
'043Cassidy 1':['043Cassidy', ],
'043Garrison/Chadwick/South Linn/Seneca':[ ],
'043East Benton/Bruner':[ ],
'043Lead Hill/Oldfield':[ ],
'043South Galloway/Saddlebrooke':[ ],
'043West Benton/Linden':[ ],
'051JC-Ward 2 Precinct 2':[ ],
'051Clark Township Brazito':[ ],
'051Clark Township Eugene':[ ],
'051Jefferson Township Apache Flats':[ ],
'051Jefferson Township CC General':[ ],
'051Jefferson Township Expressway Rental':[ ],
'051Jefferson Township JC Fair':[ ],
'051Jefferson Township Scott Station':[ ],
'051Liberty Township Taos':[ ],
'051Marion Township Centertown':[ ],
'051Marion Township St Martins':[ ],
'051Moreau Township Lohman':[ ],
'051Moreau Township Russellville':[ ],
'051Osage Township St. Thomas':[ ],
'051Osage Township Wardsville':[ ],
'055Bourbon Community Center':[ ],
'055Cherryville Fire Station':[ ],
'055Recklein Auditorium':[ ],
'055Davisville American Legion Hall':[ ],
'055Leasburg First Southern Baptist Church':[ ],
'055Steelville Community Building':[ ],
'055Sullivan Community Building':[ ],
'057AMERICAN LEGION':[ ],
'057ARCOLA LIONS CLUB':[ ],
'057DADEVILLE COMM BLDG':[ ],
'057EVERTON COMM BLDG':[ ],
'057LOCKWOOD LIBRARY':[ ],
'057RECORDER'S OFFICE':[ ],
'057SMALL COURTROOM':[ ],
'057SYLVANIA COMM BLDG':[ ],
'059Sherman/Miller':[ ],
'059Benton North OCL':[ ],
'059Benton South OCL':[ ],
'063Grant/Dallas/West Camden/Middle Camden':[ ],
'065Boss':[ ],
'067AMERICAN LEGION':[ ],
'067AVA COMMUNITY CENTER':[ ],
'067BETHANY BAPTIST CHURCH':[ ],
'067BRUSHYKNOB CHURCH':[ ],
'067EASTERN DO. CO. COMM. BUILDING':[ ],
'067GENTRY CHURCH':[ ],
'067GOODHOPE NAZARENE CHURCH':[ ],
'067HIGHWAY NAZARENE CHURCH':[ ],
'067PLEASANT HOME CHURCH':[ ],
'067PLEASANT RIDGE SOUTHERN BAPTIST':[ ],
'067SQUIRES COMMUNITY BUILDING':[ ],
'067STAR CHAPEL CHURCH':[ ],
'067VANZANT COMMUNITY BUILDING':[ ],
'069Campbell Ward 1/Campbell Ward 2 & Rural':[ ],
'069Malden Ward 1 & 2/Malden Ward 3 & Rural/Malden Ward 4':[ ],
'075Athens South/Athens North':[ ],
'075Cooper East/Cooper West':[ ],
'075Jackson West/Jackson East':[ ],
'079COURTHOUSE':[ ],
'079EDINBURG':[ ],
'079FIRST BAPTIST CHURCH':[ ],
'079FIRST CHRISTIAN CHURCH':[ ],
'079GALT':[ ],
'079LAREDO':[ ],
'079NCMC':[ ],
'079SPICKARD':[ ],
'079TINDALL':[ ],
'083Bearcreek/Deepwater':[ ],
'083Springfield/Tebo':[ ],
'087Central/West':[ ],
'087Northwest/West':[ ],
'087Southeast/Southwest':[ ],
'089Burton/Prairie':[ ],
'091Myatt 2/Howell No. 7':[ ],
'101Centerview/Columbus':[ ],
'101Hazel Hill/Simpson':[ ],
'101Knob Noster/Lowland':[ ],
'101NS Holden/Rose Hill/Kingsville':[ ],
'101Post Oak/Jefferson':[ ],
'101Warrensburg SE-1/Montserrat':[ ],
'101Warrensburg SE-2/NE':[ ],
'101Warrensburg SW/NW':[ ],
'115North Salem/Baker':[ ],
'115Brookfield 1&2':[ ],
'115Brookfield 3&4':[ ],
'115Clay/Parsons Creek':[ ],
'115Grantsville/Enterprise':[ ],
'129HAL ENGLAND CENTER':[ ],
'129MERCER':[ ],
'129MORGAN':[ ],
'129RAVANNA':[ ],
'133Anniston Ward 1/Ward 2/Rural':[ ],
'133Bertrand Ward 1/Ward 2/Rural':[ ],
'133Charleston Ward 1A/Ward 1B/Ward 2':[ ],
'133Charleston Ward 3/Ward 4':[ ],
'133East Prairie Ward 1/Ward 2/Ward 3':[ ],
'133East Prairie Rural/Wolf Island/Dorena':[ ],
'133Wilson City/Wyatt Ward 1/Ward 2/Rural':[ ],
'135Jamestown/Lupus':[ ],
'135Tipton/Fortuna':[ ],
'141Barnett I & II':[ ],
'141Stover North/Stover South':[ ],
'141Syracuse North & South':[ ],
'141Versailles North Excelsior & Fairgrounds North':[ ],
'141Versailles South Fairgrounds South':[ ],
'143Anderson 1&2':[ ],
'143Big Prairie 1&2':[ ],
'143Big Prairie 3':[ ],
'143Big Prairie 4':[ ],
'143Como 1&2':[ ],
'143Como 3&4':[ ],
'143Como 5':[ ],
'143Lafont 1&2&3':[ ],
'143Lewis 1&2&5':[ ],
'143Lewis 3&4':[ ],
'143New Madrid 1':[ ],
'143New Madrid 2&4&5/Hough 1/St. John 1':[ ],
'143New Madrid 3':[ ],
'143Portage 1/Lesieur 1':[ ],
'143Portage 2':[ ],
'143Portage 3':[ ],
'143West 1&2':[ ],
'143West 3&4':[ ],
'145Neosho #3':[ ],
'149Cedar Bluff/Myrtle':[ ],
'149Jeff/Couch':[ ],
'149Woodside/King/Piney No. 1/Piney No. 2':[ ],
'149Thayer No. 2/Thayer No. 1':[ ],
'1591st Ward':[ ],
'1592nd Ward':[ ],
'1593rd Ward':[ ],
'1594th Ward':[ ],
'159Smithton W BP/Smithton E & W/Lake Creek':[ ],
'161Dawson/St. James 1&2&3':[ ],
'161Doolittle':[ ],
'161Edgar Springs/Spring Creek':[ ],
'161Meramec/St. James 4&E&W':[ ],
'161Miller/Rolla North':[ ],
'161Newburg/Liberty':[ ],
'167E&W Madison/Campbell':[ ],
'167Johnson/Flemington/Campbell/Jefferson':[ ],
'167N&S Benton/S Greene':[ ],
'167N Greene/McKinley/Jefferson':[ ],
'167Northwest Marion/Cliquot':[ ],
'167West Looney/Wishart':[ ],
'171Union NE/Sherman/Richland':[ ],
'171Wilson/Union SW/Jackson':[ ],
'173Hassard':[ ],
'173Liberty':[ ],
'173Oakwood':[ ],
'173Perry':[ ],
'177Knoxville':['177Knox'],
'177Richmond 1&2':['177Ward 1','177Ward 2'],
'177Richmond 3&4':['177Ward 3','177Ward 4'],
'181Poynor/Current River/West Doniphan/Doniphan P1/Doniphan P2':[ ],
'181Gatewood/Union':[ ],
'181Harris/Varner P1':[ ],
'181Shirley/Kelly/Pine':[ ],
'181Thomas/Varner P2':[ ],
'181Flatwoods/Johnson P2/Washington':[ ],
'185Appleton':[ ],
'185Butler':[ ],
'185Collins/Washington':[ ],
'185Dallas/Doyal':[ ],
'185Monegaw/Osage':[ ],
'185Osceola/Polk':[ ],
'185Center/Roscoe':[ ],
'185Speedwell/Taber':[ ],
'187Farmington No. 4/Busiek':['187Busiek','187Farmington No. 4'],
'195Nelson':[ ],
'197Coffey/Downing':['197Coffey','197Downing'],
'197North Liberty/Coatsville/Glenwood/South Liberty':['197Coatsville','197Glenwood','197North Liberty','197South Liberty'],
'197East Prairie/West Prairie':['197East Prairie','197West Prairie'],
'203Spring Creek/Birch Tree/Bartlett':['203Spring Creek','203Bartlett','203Birch Tree'],
'203Eminence/Delaware':['203Delaware','203Eminence'],
'205BETHEL/TIGER FORK':['205Bethel/Tiger Fork',],
'205CLARENCE':[ ],
'205HUNNEWELL':[ ],
'205LEONARD':[ ],
'205SHELBINA EAST/WEST/LENTNER':[ ],
'205SHELBYVILLE/EMDEN/BLKCR/NRTH RIV':[ ],
'207Bloomfield Ward No. 1/2':['207Bloomfield Ward No. 2','207Bloomfield Ward No. 1'],
'209Ruth B Rural':[ ],
'209Ruth C Rural':[ ],
'209Sunset Cove':[ ],
'211Green City/Pennville':['211Green City','211Pennville'],
'211Taylor/Humphreys':['211Taylor','211Humphreys'],
'211Pollock/Boynton':['211Boynton','211Pollock'],
'211South Polk/Reger/Pleasant Hill':['211Reger','211South Polk','211Pleasant Hill'],
'211West Polk/Asbury':['211West Polk','211Asbury'],
   
#For 213, just make only first letters capitalized

    "223Peach Tree/Piedmont Ward 1/Ward 2":['223Peach Tree','223Piedmont Ward No. 1','223Piedmont Ward No. 2'],
    "223Wappapello/McGee/Shook":['223Wappapello','223McGee','223Shook'],
    "223Coldwater/Greenville/Silva":['223Coldwater','223Greenville','223Silva'],
    "223Cool Springs/Chaonia":['223Chaonia','223Cool Springs'],
    "223Lowndes/Hiram":['223Lowndes','223Hiram'],
    "223Patterson/Brunot/Smith Mill":['223Patterson','223Smith Mill','223Brunot'],
    "223Cascade/Clubb":['223Clubb','223Cascade'],
    "225East Ozark/Northview A":["225Northview A","225East Ozark"],
    "225Finley/Hazelwood":["225Finley","225Hazelwood"],
    "225Niangua/Union":["225Niangua","225Union"],
    "225Washington/Grant":["225Washington","225Grant"],
}



SyntaxError: invalid syntax (<ipython-input-1-1e7e666f5c55>, line 138)

In [ ]:
txt = "0191F/1J/1L"
txt.split("/")

Missouri
--------

Precinct mergers were made in the following counties to match county reporting units: 
- Andrew: Precinct mergers to match county reporting units
- Barry: Precinct mergers to match county reporting units
- Barry: Add Monett City; Align Cassville with corporate boundary
- Barton: Adjust Lamar ward boundaries to match city map
- Benton: Precinct mergers to match county reporting units
- Bollinger: Precinct mergers to match county reporting units
- Bollinger: Adjust Patton/Sedgewickville to match county GIS
- Butler: Align Poplar Bluff with corporate boundary and adjust wards to match city map
- Boone: Precinct mergers to match county reporting units
- Carroll: Precinct mergers to match county reporting units
- Carroll: Adjust Carrollton City wards to match voter file
- Carter: Precinct mergers to match county reporting units
- Cass: Precinct mergers to match county reporting units
- Cass: Align Belton, Pleasant Hill, Raymore precincts with corporate boundaries; Adjust 37 Pleasant Prairie/25 West Peculiar Rural
- Chariton: Adjust Salisbury wards to match city map
- Christian: Revise Cassidy 1/Cassidy 2; Reverse Nixa City annexation into Rosedale 1B
- Christian: Precinct mergers to match county reporting units
- Clark: Precinct mergers to match county reporting units
- Clay: Adjust Excelsior Springs, Kearney, Smithville precincts to match 2018 ward and corporate boundaries; Adjust KC 21-1/15, 21-1/17, 21-3/11, 21-4/6, 21-4/11, 21-5/7/24, 21-6/24, 21-7/20, 21-13/26, 21-14/23, 21-17/16, 21-17/26, 21-17/Platte 1, Gallatin 7/16, 10/13, 10/16, 11/15, 13/15, Kearney 3/Platte 2, Liberty 1/10, 6/13, 7/11, 11/14, Washington 1/3 to match voter file
- Clinton: Align Atchison/Concord, Jackson/Lathrop with 2018 precinct split assignments
- Cole: Precinct mergers to match county reporting units
- Cole: Adjust Marion Twp St Martins to include St Martins city in Jefferson Twp
- Crawford: Move Benton Twp section of Leasburg precinct from Recklein to Leasburg
- Crawford: Precinct mergers to match county reporting units
- Dade: Realign Greenfield wards to match municipal code
- Dallas: Split N Benton Inside/OCL, S Benton Inside/OCL; Align N/S Benton with Buffalo ward map
- Dallas: Precinct mergers to match county reporting units
- Dekalb: Precinct mergers to match county reporting units
- Dent: Split Boss precinct from R-4 precinct; Adjust R-2/R-3 boundary
- Douglas: Precinct mergers to match county reporting units
- Dunklin: Precinct mergers to match county reporting units
- Dunklin: Align Holcomb, Rives with township boundaries; Adjust Arbyrd/Senath; Align Kennett wards with city map
- Gasconade: Align Hermann, Owensville, Rosebud with corporate boundaries and ward descriptions
- Gentry: Precinct mergers to match county reporting units
- Grundy: Precinct mergers to match county reporting units
- Holt: Precinct mergers to match county reporting units
- Holt: Split West between Central/Northwest by school district
- Howard: Precinct mergers to match county reporting units
- Howell: Precinct mergers to match county reporting units
- Howell: Align West Plains precincts with 2018 corporate boundary
- Iron: Adjust Annapolis/Des Arc/Vulcan, Ironton/Pilot Knob
- Johnson: Adjust Warrensburg NE/NW boundary to match county maps
- Johnson: Precinct mergers to match county reporting units
- Lewis: Split LaBelle Twp between LaBelle FS/Lewistown FS with PLSS grid
- Lincoln: Align Moscow Mills, Troy with 2018 corporate boundaries; Adjust Briscoe/Troy Rural
- Linn: Precinct mergers to match county reporting units
- Linn: Split Brookfield Township between Brookfield 1&2/3&4
- Livingston: Align Chillicothe with 2018 corporate boundaries
- Livingston: Precinct mergers to match county reporting units
- Macon: Align Macon City wards with corporate boundaries
- Madison: Align Fredericktown wards with corporate boundaries
- Mercer: Precinct mergers to match county reporting units
- Mississippi: Precinct mergers to match county reporting units
- Mississippi: Adjust Bertrand/East Prairie Rural; Align East Prairie City with corporate boundary
- Moniteau: Precinct mergers to match county reporting units
- Montgomery: Align Bellflower/Middletown, Big Spring/Rhineland with voter file
- Morgan: Adjust Versaille North/South boundary to match county GIS
- Morgan: Precinct mergers to match county reporting units
- New Madrid: Precinct mergers to match county reporting units
- New Madrid: Add New Madrid City wards; Add Big Prairie 4 precinct; Move Portageville Ward 1 to Portage 1
- Newton: Align Staples precincts with Joplin City GIS boundaries; Align Neosho precincts with Neosho City GIS boundaries; Adjust Newtonia/Ritchey Wentworth
- Oregon: Precinct mergers to match county reporting units
- Pettis: Precinct mergers to match county reporting units
- Pettis: Adjust Sedalia City wards to match city map
- Phelps: Align St. James ward boundaries with city map
- Phelps: Precinct mergers to match county reporting units
- Polk: Precinct mergers to match county reporting units
- Polk: Split Campbell/Jefferson precincts by school district
- Putnam: Split Union Twp into NE/SW precincts to match county map
- Putnam: Precinct mergers to match county reporting units
- Pulaski: Adjust Big Piney/St. Roberts, Laquey/Waynesville
- Ralls: Precinct mergers to match county reporting units
- Ralls: Replace Center/Liberty precincts with 2010 VTDs
- Ray: Adjust Richmond City wards, Homestead Village/Wood Heights/Lawson, Richmond/Orrick
- Ray: Precinct mergers to match county reporting units
- Reynolds: Realign precincts countywide to match voter file
- Ripley: Precinct mergers to match county reporting units
- Saline: Split Nelson precinct from Hardeman precinct; Adjust Marshall Ward 2/3 boundary
- St. Charles: Adjust Brian/Ridgepoint, Civic/St. Paul, Harvester/Sycamore, Mamelle/Marina
- St. Clair: Precinct mergers to match county reporting units
- St. Francois: Precinct mergers to match county reporting units
- St. Francois: Align Desloge, Farmington, Park Hills with corporate boundaries and city ward maps; Adjust Bonne Terre/Timberline, Patterson/Timberline, Leadwood/Park Hills to match voter file
- St. Louis: Precinct mergers to match county reporting units
- St. Louis: Add UNV044 precinct; Split NRW011/NRW052
- Schuyler: Precinct mergers to match county reporting units
- Shannon: Precinct mergers to match county reporting units
- Shannon: Align all precincts with county GIS and voter file
- Shelby: Precinct mergers to match county reporting units
- Shelby: Adjust Jackson/North River to match voter file
- Stoddard: Precinct mergers to match county reporting units
- Stoddard: Align Advance, Bernie, Bloomfield, Dexter, Puxico with corporate boundaries; Adjust Advance/Leora, Bernie/Dexter/Pyle, Bloomfield/Gray Ridge
- Stone: Move Branson West from Ruth A to Ruth C; Split Ruth C Rural/Sunset Cove; Adjust Ruth B City/Rural to match county GIS
- Sullivan: Precinct mergers to match county reporting units
- Taney: Adjust Boston Center/Walnut Shade; Align Branson wards with 2018 corporate boundary
- Vernon: Align Bronaugh/Moundville, Harwood/Schell City, Deerfield/NG Armory with voter file
- Wayne: Precinct mergers to match county reporting units
- Webster: Precinct mergers to match county reporting units





### 2. 2010 Census VTD release
>Monroe County comes from the 2010 Census VTD release.

https://www.census.gov/cgi-bin/geo/shapefiles/index.php?year=2010&layergroup=Voting+Districts

In [ ]:
monroe_shape = gp.read_file("./raw-from-source/Census/2010_VTD/tl_2010_29137_vtd10/tl_2010_29137_vtd10.shp")

### 3. 2020 Census VTD release
>Platte County comes from the 2020 Census VTD release.

https://www.census.gov/geographies/mapping-files/time-series/geo/tiger-line-file.2020.html

In [ ]:
platte_shape = gp.read_file("./raw-from-source/Census/2020_VTD/tl_2020_29_all/tl_2020_29165_vtd20.shp")
platte_shape.plot()

- Platte: Precinct mergers to match county reporting units
- Platte: Align Lake Waukomis, Parkville, Platte City, Riverside/Northmore, Weatherby Lake with corporate boundaries; Adjust Platte Hills/Park Hill, Seven Bridges/Shiloh

### 4. Counties themselves
> The following counties instead used maps or shapefiles sourced from the respective county governments:  
Audrain, Bates, Caldwell, Callaway, Camden, Cape Girardeau, Cedar, Cooper, Franklin, Greene, Jackson, Jasper, Jefferson, Laclede, Lafayette, Lawrence, Marion, McDonald, Miller, Nodaway, Osage, Ozark, Pemiscot, Pike, Randolph, Scott, Ste. Genevieve, Texas, Warren, Washington, Worth, Wright.  

- Audrain: https://audraingis.integritygis.com/H5/Index.html?viewer=audrain
- Bates: https://batesgis.integritygis.com/H5/Index.html?viewer=bates
- Caldwell: can't find
- Callaway: can't find
- Camden: https://camdengis.integritygis.com/H5/Index.html?viewer=camden
- Cape Girardeau: only pdf (https://capecountyelections.com/wp-content/uploads/2019/06/Voter_Precincts.pdf)
- Cedar: can't find
- Cooper: https://coopergis.integritygis.com/H5/Index.html?viewer=cooper
- Franklin: can't find
- Greene: not available (https://www.greenecountyassessor.org/assessor/maps/mapadv.aspx)
- Jackson: not downloadable (https://jcebmo.org/wp-content/uploads/JACKSON-COUNTY-TOWNSHIPS-PRECINCTS.pdf)
- Jasper: https://jaspercountymogisintiatives-jcmo.hub.arcgis.com/datasets/voting-precincts-2020
- Jefferson: can't find
- Laclede: https://lacledegis.integritygis.com/H5/Index.html?viewer=laclede
- Lafayette: https://lafayettegis.integritygis.com/H5/Index.html?viewer=lafayette
- Lawrence: not downloadable (https://www.lawrencecountymo.org/elections)
- Marion: https://mariongis.integritygis.com/H5/Index.html?viewer=marion_public
- McDonald: can't find
- Miller: not available (https://millergis.integritygis.com/H5/Index.html?viewer=miller)
- Nodaway: can't find
- Osage: can't find
- Ozark: can't find
- Pemiscot: can't find
- Pike: can't find
- Randolph: can't find
- Scott: not downloadable https://www.arcgis.com/apps/webappviewer/index.html?id=73de0cc0034040a5b8b0f5988a643164
- Ste. Genevieve: can't find
- Texas: can't find
- Warren: not downloadable (https://www.arcgis.com/apps/webappviewer/index.html?id=edfaf72f348b4ea28a959815d82dfe86)
- Washington: can't find
- Worth: can't find
- Wright: can't find

### Shapefile Modifications

- Audrain: Precinct mergers to match county reporting units
- Audrain: Add Vandalia City; Align Mexico precincts with municipal code
  
- Callaway: Align Fulton, Holts Summit precincts with 2018 corporate boundaries
- Callaway: Precinct mergers to match county reporting units

- Camden: Precinct mergers to match county reporting units

- Cape Girardeau: Precinct mergers to match county reporting units

- Cooper: Precinct mergers to match county reporting units

- Franklin: Split Dry Branch between Stanton & St. Clair Out of Town
- Franklin: Precinct mergers to match county reporting units

- Greene: Precinct mergers to match county reporting units

- Jackson: Precinct mergers to match county reporting units

- Jasper: Precinct mergers to match county reporting units
- Jasper: Realign Carthage 2/3, Joplin 1/7, 10/13, 10/15, 13/22, Oronogo 1/2, Webb City 1/4 to match city ward maps

- Jefferson: Precinct mergers to match county reporting units
- Jefferson: Split Festus Ward 1 P1/P2, Ward 3 P1/P2, Ward 4 P1/P2; Adjust Crystal City/Riverview

- Marion: Precinct mergers to match county reporting units
- Marion: Split Fabius-Smileyville/Taylor, Warren-Monroe City/Warren; Align Hannibal wards with voter file

- Miller: Adjust all precinct boundaries to match voter file

- Nodaway: Precinct mergers to match county reporting units

- Osage: Precinct mergers to match county reporting units

- Pemiscot: Precinct mergers to match county reporting units

- Pike: Adjust Cuivre JL boundaries with Ashley NO, Buffalo E, Prairieville I

- Randolph: Precinct mergers to match county reporting units

- Scott: Adjust Haywood City/Morley, Sikeston Ward 1/McMullin/Vanduser

- Warren: Precinct mergers to match county reporting units

- Washington: Adjust City Hall/Stony Point, Cruise/Richwoods to match county GIS

- Worth: Precinct mergers to match county reporting units